# Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
from src.read_data import read_file
from src.text_normalization import arabic_text_normalization
from src.text_processing import data_words, words_to_index_, index_to_word_, word_counts, text_to_sequence, sequences_padding, word_sequence_to_text
import random
from tensorflow.keras.layers import Attention, AdditiveAttention, MultiHeadAttention # type: ignore
from tensorflow.keras.layers import LSTM, GRU, Bidirectional# type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore
from src.data_split import input_output_split, decoder_input_output
from src.encoder import encoder_without_attention, encoder_with_attention
from src.decoder import decoder_without_attention, decoder_with_attention
from src.encoder_decoder_model import build_model, model_compile, model_fit, save_model, summary
from src.transformers import transformers
import warnings
import os
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Ignore Warnings

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings(action='ignore')

# Set Early Stop For Encoder-Decoder Model

In [3]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

# Read Data

In [6]:
Corpus = read_file(file_path='Question Answer Data.txt')
Corpus = [sentences for sentences in Corpus if len(sentences) < 500]
len(Corpus)

2221

In [7]:
random.seed(44)
Corpus = random.sample(Corpus, len(Corpus))

# Split The Data Into Encoder Input And Output

In [8]:
context, question, answer = input_output_split(data=Corpus, text_normalization1=arabic_text_normalization, text_normalization2=arabic_text_normalization, text_normalization3 =arabic_text_normalization)

# Text Processing

**1-Word To Integer (Tokenizer)**

In [9]:
all_words , words = data_words(filter='"#$&*+/:=@[\\]^_{|}~', data=(context + question + answer)) # Set With Unique Words And Find Number Of Unique Words
words_to_index = words_to_index_(words=words) # Convert Word To Integer Index
index_to_words = index_to_word_(words=words)

In [10]:
all_words

19473

**2-Data To Sequence**

In [11]:
context_sequences = text_to_sequence(word_index=words_to_index, data=context) # Convert Text Into Squences Of Integer
question_sequences = text_to_sequence(word_index=words_to_index, data=question) # Convert Text Into Squences Of Integer
answer_sequences = text_to_sequence(word_index=words_to_index, data=answer) # Convert Text Into Squences Of Integer

**3-Max Length**

In [12]:
context_max_length = len(max(context_sequences, key = len)) # Find Max Length
question_max_length = len(max(question_sequences, key = len))
max_length = max(context_max_length, question_max_length)
answer_max_length = len(max(answer_sequences, key=len))# Find Max Length

In [13]:
max_length

212

**4-Encoder Zero Padding**

In [14]:
pad_context = sequences_padding(input_sequence=context_sequences, max_length=max_length) # Zero Padding (Add Zeros To End Of Sequence In Input Sequences To Make All Sequences In Same Length)
pad_question = sequences_padding(input_sequence=question_sequences, max_length=max_length) # Zero Padding (Add Zeros To End Of Sequence In Input Sequences To Make All Sequences In Same Length)

**5-Generate Decoder Input And Output**

In [15]:
decoder_input, decoder_output = decoder_input_output(output_data= answer_sequences)

**6-Decoder Zero Padding**

In [16]:
pad_decoder_input = sequences_padding(input_sequence=decoder_input, max_length=answer_max_length) # Zero Padding (Add Zeros To End Of Sequence In Input Sequences To Make All Sequences In Same Length)
pad_decoder_output = sequences_padding(input_sequence=decoder_output, max_length=answer_max_length) # Zero Padding (Add Zeros To End Of Sequence In Input Sequences To Make All Sequences In Same Length)

**7-Word Occurrence**

In [17]:
word_count = word_counts(data=(context + question + answer)) # Number Of Occurrence Each Word

# Encoder-Decoder Without Attention

In [18]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

**1-LSTM**

In [19]:
encoder_model_1 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states = encoder_without_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_1)

decoder_model_1 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_without_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_1)

model_1 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_1)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, None, 200) │          0 │ embedding[0][0],  │
│ (Concatenate)       │                   │            │ embedding[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 128) │  2,492,544 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, None,     │    168,448 │ concatenate[0][0] │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │    131,584 │ embedding_1[0][0… │
│                     │ 128), (None,      │            │ lstm[0][1],       │
│                     │ 128), (None,      │            │ lstm[0][2]        │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │  2,512,017 │ lstm_1[0][0]      │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,251,893 (27.66 MB)

 Trainable params: 7,251,893 (27.66 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model_compile(model = model_1, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_1, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_1,name="LSTM_1")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 25s 339ms/step - loss: 8.4459 - sparse_categorical_accuracy: 0.0251
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 25s 351ms/step - loss: 5.3937 - sparse_categorical_accuracy: 0.0270
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 30s 422ms/step - loss: 5.1739 - sparse_categorical_accuracy: 0.0270
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 26s 365ms/step - loss: 5.0157 - sparse_categorical_accuracy: 0.0285
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 26s 371ms/step - loss: 4.8596 - sparse_categorical_accuracy: 0.0303
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 26s 366ms/step - loss: 4.7291 - sparse_categorical_accuracy: 0.0303
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 25s 363ms/step - loss: 4.6215 - sparse_categorical_accuracy: 0.0304
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 25s 359ms/step - loss: 4.5356 - sparse_categorical_accuracy: 0.0307
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 25s 356ms/step - loss: 4.4623 - sparse_categorical_accuracy: 0.0310
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━

In [21]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

**2-GRU**

In [22]:
encoder_model_2 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states = encoder_without_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_2)

decoder_model_2 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_without_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_2)

model_2 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_2)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, None, 200) │          0 │ embedding_2[0][0… │
│ (Concatenate)       │                   │            │ embedding_2[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 128) │  2,492,544 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru (GRU)           │ [(None, None,     │    126,720 │ concatenate_1[0]… │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ [(None, None,     │     99,072 │ embedding_3[0][0… │
│                     │ 128), (None,      │            │ gru[0][1]         │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None,      │  2,512,017 │ gru_1[0][0]       │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,177,653 (27.38 MB)

 Trainable params: 7,177,653 (27.38 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model_compile(model = model_2, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_2, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_2, name="GRU_1")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 24s 315ms/step - loss: 8.4798 - sparse_categorical_accuracy: 0.0245
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 22s 316ms/step - loss: 5.3810 - sparse_categorical_accuracy: 0.0270
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 303ms/step - loss: 5.1610 - sparse_categorical_accuracy: 0.0303
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 22s 315ms/step - loss: 5.0516 - sparse_categorical_accuracy: 0.0307
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 22s 314ms/step - loss: 4.9361 - sparse_categorical_accuracy: 0.0308
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 22s 313ms/step - loss: 4.8199 - sparse_categorical_accuracy: 0.0309
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 23s 326ms/step - loss: 4.6860 - sparse_categorical_accuracy: 0.0309
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 22s 308ms/step - loss: 4.5601 - sparse_categorical_accuracy: 0.0326
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 24s 336ms/step - loss: 4.4517 - sparse_categorical_accuracy: 0.0336
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━

In [24]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

**3-Bidirectional**

In [25]:
encoder_model_3 = Bidirectional(LSTM(units=128, return_sequences=True, return_state=True, seed=33))
context_inputs, question_inputs, encoder_states = encoder_without_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_3)

decoder_model_3 = LSTM(units=256, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_without_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_3)

model_3 =  build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_3)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, None, 200) │          0 │ embedding_4[0][0… │
│ (Concatenate)       │                   │            │ embedding_4[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, None,     │    336,896 │ concatenate_2[0]… │
│ (Bidirectional)     │ 256), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, None, 128) │  2,492,544 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 256)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 256)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, None,     │    394,240 │ embedding_5[0][0… │
│                     │ 256), (None,      │            │ concatenate_3[0]… │
│                     │ 256), (None,      │            │ concatenate_4[0]… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, None,      │  5,004,561 │ lstm_3[0][0]      │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,175,541 (38.82 MB)

 Trainable params: 10,175,541 (38.82 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model_compile(model = model_3, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_3, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_3,name="Bidirectional_1")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 35s 470ms/step - loss: 8.0186 - sparse_categorical_accuracy: 0.0252
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 42s 600ms/step - loss: 5.3054 - sparse_categorical_accuracy: 0.0271
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 43s 615ms/step - loss: 5.0244 - sparse_categorical_accuracy: 0.0291
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 39s 549ms/step - loss: 4.6653 - sparse_categorical_accuracy: 0.0308
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 43s 608ms/step - loss: 4.4053 - sparse_categorical_accuracy: 0.0329
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 38s 539ms/step - loss: 4.2199 - sparse_categorical_accuracy: 0.0342
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 42s 605ms/step - loss: 4.0874 - sparse_categorical_accuracy: 0.0349
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 40s 566ms/step - loss: 3.9881 - sparse_categorical_accuracy: 0.0356
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 39s 554ms/step - loss: 3.8692 - sparse_categorical_accuracy: 0.0362
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━

# Encoder-Decoder With Attention

**1-Luong Attension**

In [27]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

*A-LSTM*

In [28]:
encoder_model_4 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_4)

attention_layer = Attention(use_scale=True)
decoder_model_4 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_4, encoder_outputs=encoder_outputs, attention_layer=attention_layer)

model_4 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_4)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, None, 200) │          0 │ embedding_6[0][0… │
│ (Concatenate)       │                   │            │ embedding_6[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_7         │ (None, None, 128) │  2,492,544 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, None,     │    168,448 │ concatenate_5[0]… │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, None,     │    131,584 │ embedding_7[0][0… │
│                     │ 128), (None,      │            │ lstm_4[0][1],     │
│                     │ 128), (None,      │            │ lstm_4[0][2]      │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, None, 128) │          1 │ lstm_5[0][0],     │
│ (Attention)         │                   │            │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, None, 256) │          0 │ lstm_5[0][0],     │
│ (Concatenate)       │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, None,      │  5,004,561 │ concatenate_6[0]… │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,744,438 (37.17 MB)

 Trainable params: 9,744,438 (37.17 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
model_compile(model = model_4, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_4, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_4, name="LSTM_2")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 33s 453ms/step - loss: 5.0237 - sparse_categorical_accuracy: 0.8348
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 34s 490ms/step - loss: 0.6050 - sparse_categorical_accuracy: 0.9210
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 32s 456ms/step - loss: 0.5702 - sparse_categorical_accuracy: 0.9226
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 35s 501ms/step - loss: 0.5549 - sparse_categorical_accuracy: 0.9260
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 31s 450ms/step - loss: 0.5389 - sparse_categorical_accuracy: 0.9260
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 31s 438ms/step - loss: 0.5159 - sparse_categorical_accuracy: 0.9260
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 36s 505ms/step - loss: 0.4956 - sparse_categorical_accuracy: 0.9269
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 32s 450ms/step - loss: 0.4801 - sparse_categorical_accuracy: 0.9281
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 34s 489ms/step - loss: 0.4690 - sparse_categorical_accuracy: 0.9287
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━

*B-GRU*

In [30]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [31]:
encoder_model_5 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_5)

attention_layer = Attention(use_scale=True)
decoder_model_5 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_5, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_5 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_5)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_8         │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_7       │ (None, None, 200) │          0 │ embedding_8[0][0… │
│ (Concatenate)       │                   │            │ embedding_8[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_9         │ (None, None, 128) │  2,492,544 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_2 (GRU)         │ [(None, None,     │    126,720 │ concatenate_7[0]… │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_3 (GRU)         │ [(None, None,     │     99,072 │ embedding_9[0][0… │
│                     │ 128), (None,      │            │ gru_2[0][1]       │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_1         │ (None, None, 128) │          1 │ gru_3[0][0],      │
│ (Attention)         │                   │            │ gru_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_8       │ (None, None, 256) │          0 │ gru_3[0][0],      │
│ (Concatenate)       │                   │            │ attention_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, None,      │  5,004,561 │ concatenate_8[0]… │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,670,198 (36.89 MB)

 Trainable params: 9,670,198 (36.89 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
model_compile(model = model_5, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_5, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_5, name="GRU_2")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 31s 414ms/step - loss: 4.8205 - sparse_categorical_accuracy: 0.8364
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 415ms/step - loss: 0.5891 - sparse_categorical_accuracy: 0.9218
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 417ms/step - loss: 0.5524 - sparse_categorical_accuracy: 0.9260
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 411ms/step - loss: 0.5196 - sparse_categorical_accuracy: 0.9262
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 414ms/step - loss: 0.4935 - sparse_categorical_accuracy: 0.9270
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 413ms/step - loss: 0.4715 - sparse_categorical_accuracy: 0.9286
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 412ms/step - loss: 0.4582 - sparse_categorical_accuracy: 0.9292
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 416ms/step - loss: 0.4473 - sparse_categorical_accuracy: 0.9296
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 34s 489ms/step - loss: 0.4385 - sparse_categorical_accuracy: 0.9300
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━

*C-Bidirectional*

In [33]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [34]:
encoder_model_6 = Bidirectional(LSTM(units=128, return_sequences=True, return_state=True, seed=33))
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_6)

attention_layer = Attention(use_scale=True)
decoder_model_6 = LSTM(units=256, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_6, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_6 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_6)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_10        │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_9       │ (None, None, 200) │          0 │ embedding_10[0][… │
│ (Concatenate)       │                   │            │ embedding_10[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ [(None, None,     │    336,896 │ concatenate_9[0]… │
│ (Bidirectional)     │ 256), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, None, 128) │  2,492,544 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_10      │ (None, 256)       │          0 │ bidirectional_1[… │
│ (Concatenate)       │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_11      │ (None, 256)       │          0 │ bidirectional_1[… │
│ (Concatenate)       │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ [(None, None,     │    394,240 │ embedding_11[0][… │
│                     │ 256), (None,      │            │ concatenate_10[0… │
│                     │ 256), (None,      │            │ concatenate_11[0… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_2         │ (None, None, 256) │          1 │ lstm_7[0][0],     │
│ (Attention)         │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_12      │ (None, None, 512) │          0 │ lstm_7[0][0],     │
│ (Concatenate)       │                   │            │ attention_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None,      │  9,989,649 │ concatenate_12[0… │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,160,630 (57.83 MB)

 Trainable params: 15,160,630 (57.83 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
model_compile(model = model_6, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_6, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_6, name="Bidirectional_2")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 58s 798ms/step - loss: 3.8940 - sparse_categorical_accuracy: 0.8355
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 67s 963ms/step - loss: 0.5950 - sparse_categorical_accuracy: 0.9222
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 67s 957ms/step - loss: 0.5712 - sparse_categorical_accuracy: 0.9260
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 63s 896ms/step - loss: 0.5577 - sparse_categorical_accuracy: 0.9260
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 64s 918ms/step - loss: 0.5447 - sparse_categorical_accuracy: 0.9260
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 66s 944ms/step - loss: 0.5261 - sparse_categorical_accuracy: 0.9262
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 65s 920ms/step - loss: 0.5029 - sparse_categorical_accuracy: 0.9265
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 69s 984ms/step - loss: 0.4836 - sparse_categorical_accuracy: 0.9276
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - loss: 0.4701 - sparse_categorical_accuracy: 0.9282
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 

**2-Bahdanau Attension**

*A-LSTM*

In [36]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [37]:
encoder_model_7 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_7)

attention_layer = AdditiveAttention(use_scale=True)
decoder_model_7 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_7, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_7 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_7)

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_12        │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_13      │ (None, None, 200) │          0 │ embedding_12[0][… │
│ (Concatenate)       │                   │            │ embedding_12[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_13        │ (None, None, 128) │  2,492,544 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_8 (LSTM)       │ [(None, None,     │    168,448 │ concatenate_13[0… │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_9 (LSTM)       │ [(None, None,     │    131,584 │ embedding_13[0][… │
│                     │ 128), (None,      │            │ lstm_8[0][1],     │
│                     │ 128), (None,      │            │ lstm_8[0][2]      │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ additive_attention  │ (None, None, 128) │        128 │ lstm_9[0][0],     │
│ (AdditiveAttention) │                   │            │ lstm_8[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_14      │ (None, None, 256) │          0 │ lstm_9[0][0],     │
│ (Concatenate)       │                   │            │ additive_attenti… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, None,      │  5,004,561 │ concatenate_14[0… │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,744,565 (37.17 MB)

 Trainable params: 9,744,565 (37.17 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model_compile(model = model_7, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_7, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_7, name="LSTM_3")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 40s 552ms/step - loss: 5.0265 - sparse_categorical_accuracy: 0.8353
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 38s 539ms/step - loss: 0.6036 - sparse_categorical_accuracy: 0.9212
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 37s 528ms/step - loss: 0.5715 - sparse_categorical_accuracy: 0.9227
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 40s 566ms/step - loss: 0.5559 - sparse_categorical_accuracy: 0.9248
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 40s 574ms/step - loss: 0.5419 - sparse_categorical_accuracy: 0.9260
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 35s 491ms/step - loss: 0.5254 - sparse_categorical_accuracy: 0.9260
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 35s 505ms/step - loss: 0.5094 - sparse_categorical_accuracy: 0.9268
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 38s 542ms/step - loss: 0.4959 - sparse_categorical_accuracy: 0.9273
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 39s 556ms/step - loss: 0.4835 - sparse_categorical_accuracy: 0.9276
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━

*B-GRU*

In [39]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [40]:
encoder_model_8 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_8)

attention_layer = Attention(use_scale=True)
decoder_model_8 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_8, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_8 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_8)

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_14        │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_15      │ (None, None, 200) │          0 │ embedding_14[0][… │
│ (Concatenate)       │                   │            │ embedding_14[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_15        │ (None, None, 128) │  2,492,544 │ input_layer_7[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_4 (GRU)         │ [(None, None,     │    126,720 │ concatenate_15[0… │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_5 (GRU)         │ [(None, None,     │     99,072 │ embedding_15[0][… │
│                     │ 128), (None,      │            │ gru_4[0][1]       │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_3         │ (None, None, 128) │          1 │ gru_5[0][0],      │
│ (Attention)         │                   │            │ gru_4[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_16      │ (None, None, 256) │          0 │ gru_5[0][0],      │
│ (Concatenate)       │                   │            │ attention_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, None,      │  5,004,561 │ concatenate_16[0… │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,670,198 (36.89 MB)

 Trainable params: 9,670,198 (36.89 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
model_compile(model = model_8, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_8, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_8, name="GRU_3")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 30s 406ms/step - loss: 4.8317 - sparse_categorical_accuracy: 0.8363
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 32s 458ms/step - loss: 0.5888 - sparse_categorical_accuracy: 0.9218
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 33s 463ms/step - loss: 0.5552 - sparse_categorical_accuracy: 0.9261
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 34s 486ms/step - loss: 0.5206 - sparse_categorical_accuracy: 0.9262
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 31s 445ms/step - loss: 0.4914 - sparse_categorical_accuracy: 0.9266
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 30s 425ms/step - loss: 0.4708 - sparse_categorical_accuracy: 0.9282
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 34s 488ms/step - loss: 0.4578 - sparse_categorical_accuracy: 0.9287
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 30s 429ms/step - loss: 0.4478 - sparse_categorical_accuracy: 0.9291
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 35s 496ms/step - loss: 0.4442 - sparse_categorical_accuracy: 0.9297
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━

*C-Bidirectional*

In [42]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [43]:
encoder_model_9 = Bidirectional(LSTM(units=128, return_sequences=True, return_state=True, seed=33))
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_9)

attention_layer = AdditiveAttention(use_scale=True)
decoder_model_9 = LSTM(units=256, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_9, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_9 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_9)

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_16        │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_17      │ (None, None, 200) │          0 │ embedding_16[0][… │
│ (Concatenate)       │                   │            │ embedding_16[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_2     │ [(None, None,     │    336,896 │ concatenate_17[0… │
│ (Bidirectional)     │ 256), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_17        │ (None, None, 128) │  2,492,544 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_18      │ (None, 256)       │          0 │ bidirectional_2[… │
│ (Concatenate)       │                   │            │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_19      │ (None, 256)       │          0 │ bidirectional_2[… │
│ (Concatenate)       │                   │            │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_11 (LSTM)      │ [(None, None,     │    394,240 │ embedding_17[0][… │
│                     │ 256), (None,      │            │ concatenate_18[0… │
│                     │ 256), (None,      │            │ concatenate_19[0… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ additive_attention… │ (None, None, 256) │        256 │ lstm_11[0][0],    │
│ (AdditiveAttention) │                   │            │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_20      │ (None, None, 512) │          0 │ lstm_11[0][0],    │
│ (Concatenate)       │                   │            │ additive_attenti… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, None,      │  9,989,649 │ concatenate_20[0… │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,160,885 (57.83 MB)

 Trainable params: 15,160,885 (57.83 MB)

 Non-trainable params: 0 (0.00 B)

In [44]:
model_compile(model = model_9, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_9, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_9, name="Bidirectional_3")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - loss: 3.9750 - sparse_categorical_accuracy: 0.8367
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - loss: 0.5924 - sparse_categorical_accuracy: 0.9225
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - loss: 0.5670 - sparse_categorical_accuracy: 0.9260
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - loss: 0.5482 - sparse_categorical_accuracy: 0.9260
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - loss: 0.5155 - sparse_categorical_accuracy: 0.9261
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - loss: 0.4897 - sparse_categorical_accuracy: 0.9272
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - loss: 0.4688 - sparse_categorical_accuracy: 0.9279
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - loss: 0.4512 - sparse_categorical_accuracy: 0.9282
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - loss: 0.4356 - sparse_categorical_accuracy: 0.9289
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - loss: 0.42

**3-MultiHead Attesnion**

*A-LSTM*

In [45]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [46]:
encoder_model_10 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_10)

attention_layer = MultiHeadAttention(num_heads=4, key_dim=32)
decoder_model_10 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model= decoder_model_10, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_10 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_10)

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_18        │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_21      │ (None, None, 200) │          0 │ embedding_18[0][… │
│ (Concatenate)       │                   │            │ embedding_18[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_19        │ (None, None, 128) │  2,492,544 │ input_layer_9[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_12 (LSTM)      │ [(None, None,     │    168,448 │ concatenate_21[0… │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_13 (LSTM)      │ [(None, None,     │    131,584 │ embedding_19[0][… │
│                     │ 128), (None,      │            │ lstm_12[0][1],    │
│                     │ 128), (None,      │            │ lstm_12[0][2]     │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ [(None, None,     │     66,048 │ lstm_12[0][0],    │
│ (MultiHeadAttentio… │ 128), (None, 4,   │            │ lstm_13[0][0],    │
│                     │ None, None)]      │            │ lstm_12[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_22      │ (None, None, 256) │          0 │ lstm_13[0][0],    │
│ (Concatenate)       │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, None,      │  5,004,561 │ concatenate_22[0… │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,810,485 (37.42 MB)

 Trainable params: 9,810,485 (37.42 MB)

 Non-trainable params: 0 (0.00 B)

In [47]:
model_compile(model = model_10, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_10, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_10, name="LSTM_4")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 33s 449ms/step - loss: 4.6912 - sparse_categorical_accuracy: 0.8363
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 31s 439ms/step - loss: 0.5764 - sparse_categorical_accuracy: 0.9225
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 31s 440ms/step - loss: 0.5464 - sparse_categorical_accuracy: 0.9227
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 31s 437ms/step - loss: 0.5228 - sparse_categorical_accuracy: 0.9227
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 31s 445ms/step - loss: 0.5013 - sparse_categorical_accuracy: 0.9253
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 31s 441ms/step - loss: 0.4791 - sparse_categorical_accuracy: 0.9274
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 36s 516ms/step - loss: 0.4653 - sparse_categorical_accuracy: 0.9278
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 31s 443ms/step - loss: 0.4563 - sparse_categorical_accuracy: 0.9283
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 30s 434ms/step - loss: 0.4438 - sparse_categorical_accuracy: 0.9292
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━

*B-GRU*

In [48]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [49]:
encoder_model_11= GRU(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_11)

attention_layer = MultiHeadAttention(num_heads=4, key_dim=32)
decoder_model_11 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_11, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_11 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_11)

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_20        │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_10      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_23      │ (None, None, 200) │          0 │ embedding_20[0][… │
│ (Concatenate)       │                   │            │ embedding_20[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_21        │ (None, None, 128) │  2,492,544 │ input_layer_10[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_6 (GRU)         │ [(None, None,     │    126,720 │ concatenate_23[0… │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_7 (GRU)         │ [(None, None,     │     99,072 │ embedding_21[0][… │
│                     │ 128), (None,      │            │ gru_6[0][1]       │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ [(None, None,     │     66,048 │ gru_6[0][0],      │
│ (MultiHeadAttentio… │ 128), (None, 4,   │            │ gru_7[0][0],      │
│                     │ None, None)]      │            │ gru_6[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_24      │ (None, None, 256) │          0 │ gru_7[0][0],      │
│ (Concatenate)       │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, None,      │  5,004,561 │ concatenate_24[0… │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,736,245 (37.14 MB)

 Trainable params: 9,736,245 (37.14 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
model_compile(model = model_11, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_11, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_11, name="GRU_4")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 34s 454ms/step - loss: 4.5512 - sparse_categorical_accuracy: 0.8416
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 30s 424ms/step - loss: 0.5685 - sparse_categorical_accuracy: 0.9230
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 421ms/step - loss: 0.5357 - sparse_categorical_accuracy: 0.9259
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 30s 428ms/step - loss: 0.5130 - sparse_categorical_accuracy: 0.9256
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 417ms/step - loss: 0.4875 - sparse_categorical_accuracy: 0.9269
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 410ms/step - loss: 0.4678 - sparse_categorical_accuracy: 0.9273
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 411ms/step - loss: 0.4557 - sparse_categorical_accuracy: 0.9279
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 411ms/step - loss: 0.4387 - sparse_categorical_accuracy: 0.9288
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 413ms/step - loss: 0.4326 - sparse_categorical_accuracy: 0.9296
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━

*C-Bidirectional*

In [51]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [52]:
encoder_model_12 = Bidirectional(LSTM(units=128, return_sequences=True, return_state=True, seed=33))
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_12)

attention_layer = MultiHeadAttention(num_heads=4, key_dim=32)
decoder_model_12 = LSTM(units=256, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_12, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_12 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_12)

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_22        │ (None, None, 100) │  1,947,300 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_25      │ (None, None, 200) │          0 │ embedding_22[0][… │
│ (Concatenate)       │                   │            │ embedding_22[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_11      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_3     │ [(None, None,     │    336,896 │ concatenate_25[0… │
│ (Bidirectional)     │ 256), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_23        │ (None, None, 128) │  2,492,544 │ input_layer_11[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_26      │ (None, 256)       │          0 │ bidirectional_3[… │
│ (Concatenate)       │                   │            │ bidirectional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_27      │ (None, 256)       │          0 │ bidirectional_3[… │
│ (Concatenate)       │                   │            │ bidirectional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_15 (LSTM)      │ [(None, None,     │    394,240 │ embedding_23[0][… │
│                     │ 256), (None,      │            │ concatenate_26[0… │
│                     │ 256), (None,      │            │ concatenate_27[0… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ [(None, None,     │    131,712 │ bidirectional_3[… │
│ (MultiHeadAttentio… │ 256), (None, 4,   │            │ lstm_15[0][0],    │
│                     │ None, None)]      │            │ bidirectional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_28      │ (None, None, 512) │          0 │ lstm_15[0][0],    │
│ (Concatenate)       │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, None,      │  9,989,649 │ concatenate_28[0… │
│                     │ 19473)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,292,341 (58.34 MB)

 Trainable params: 15,292,341 (58.34 MB)

 Non-trainable params: 0 (0.00 B)

In [53]:
model_compile(model = model_12, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_12, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_12, name="Bidirectional_4")

Epoch 1/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 61s 840ms/step - loss: 3.8850 - sparse_categorical_accuracy: 0.8341
Epoch 2/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 67s 954ms/step - loss: 0.5777 - sparse_categorical_accuracy: 0.9228
Epoch 3/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 69s 980ms/step - loss: 0.5414 - sparse_categorical_accuracy: 0.9260
Epoch 4/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 66s 949ms/step - loss: 0.5098 - sparse_categorical_accuracy: 0.9260
Epoch 5/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 69s 990ms/step - loss: 0.4861 - sparse_categorical_accuracy: 0.9272
Epoch 6/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 68s 974ms/step - loss: 0.4678 - sparse_categorical_accuracy: 0.9283
Epoch 7/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 63s 891ms/step - loss: 0.4501 - sparse_categorical_accuracy: 0.9289
Epoch 8/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 67s 963ms/step - loss: 0.4334 - sparse_categorical_accuracy: 0.9291
Epoch 9/120
70/70 ━━━━━━━━━━━━━━━━━━━━ 67s 952ms/step - loss: 0.4201 - sparse_categorical_accuracy: 0.9294
Epoch 10/120
70/70 ━━━━━━━━━━━━━━━━━━

# Transformers

In [54]:
context = 'نظرا لأنه تم تعيينها كعاصمة وطنية تم بناء العديد من الهياكل في ذلك الوقت حتى اليوم لا يزال بعضها مفتوحا للسياح'
question = 'ماذا حدث في الوقت الذي تم فيه تعيين نانجينغ كعاصمة ؟ '
model_name = "wonfs/arabert-v2-qa"

In [55]:
transformers(model_name=model_name, context=context, question=question)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at wonfs/arabert-v2-qa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.



Context: نظرا لأنه تم تعيينها كعاصمة وطنية تم بناء العديد من الهياكل في ذلك الوقت حتى اليوم لا يزال بعضها مفتوحا للسياح
Question: ماذا حدث في الوقت الذي تم فيه تعيين نانجينغ كعاصمة ؟ 
Answer: تم بناء العديد من الهياكل في ذلك الوقت
